In [51]:
"""
Author: Yanxi Lu

This file implements the GMN model proposed in the paper "Graph Matching Network for Learning the similarity of Graph
Structured Objects.
"""

'\nAuthor: Yanxi Lu\n\nThis file implements the GMN model proposed in the paper "Graph Matching Network for Learning the similarity of Graph\nStructured Objects.\n'

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable

In [53]:
"""
Graph generation
"""
# simplification: assume all nodes are connected to each other, node feature dim = edge feature dim
def generate_one_graph(node_num, node_feats, edge_feats):

    graph1node = []
    for i in range(node_num):
        graph1node.append([1] * node_feats)

    compression_list = []
    graph1edge = []
    for i in range(node_num):
        graph1edge.append([])
        compression_list.append(-1)
        for j in range(node_num):
            graph1edge[i].append([1]*edge_feats)
            compression_list[i] += 1
    
    # Calculate the matching matrix for the use of message passing stage. Because each node may receive message from
    # all their neighbours. This matrix can be used for all propagation layers. It is ordered.
    adj_matrix = np.eye(node_num, node_num)
    adj_matrix = np.ones((node_num, node_num)) - adj_matrix
    
    matching_matrix = []
    for n in range(len(adj_matrix)):
        for m in range(len(adj_matrix[n])):
            if adj_matrix[n][m] == 1:
                matching_matrix.append((n, m))
                
    return graph1node, graph1edge, adj_matrix, matching_matrix, compression_list
graph, edge, adj_matrix, matching_matrix, compression_list = generate_one_graph(25, 25, 4)
graph1, edge1, adj_matrix1, matching_matrix1, compression_list1 = generate_one_graph(5, 25, 5)
graph1, edge1, adj_matrix1, matching_matrix1, compression_list1

([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 [[[1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1]],
  [[1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1]],
  [[1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1]],
  [[1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1]],
  [[1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1],
   [1, 1, 1, 1, 1]]],
 array([[0., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1.],
        [1., 1., 0., 1., 1.],
        

In [54]:
# Use the matching matrix to construct the concatenated input for message layer for both graphs
def match(hn, edge, hn1, edge1):
    result = torch.tensor([])
    for (i, j) in matching_matrix:
        entry = torch.cat((hn[i],hn[j],edge[i][j]),dim = 0)
        if len(result) == 0:
            result = entry.view(-1, len(entry))
        else:
            result = torch.cat((result,entry.view(-1, len(entry))),dim = 0)
    for (i, j) in matching_matrix1:
        entry = torch.cat((hn1[i],hn1[j],edge1[i][j]),dim = 0)
        if len(result) == 0:
            result = entry.view(-1, len(entry))
        else:
            result = torch.cat((result,entry.view(-1, len(entry))),dim = 0)
    return result

# compress incoming message for a node from its neighbours
def compress(msg_out, compression_list):
    result = torch.tensor([])
    for j in range(len(compression_list)):
        if j == 0:
            result = torch.sum(msg_out[:compression_list[j]], dim=0).view(-1, len(msg_out[0]))
        else:
            result = torch.cat((result, torch.sum(msg_out[:compression_list[j]], dim=0).view(-1, len(msg_out[0]))),dim = 0)
#     print(">>>>>>>>> Compression:", result)
    return result
        

# compute the result of cross-graph attention based matching mechanism, the attention is similar to soft-max, so that if there 
# is one perfect match in the other graph, the attention would be 1 as all other terms are 0.
def cross_graph_matching(hn1, hn2):
    result = torch.tensor([])
    for n1 in hn1:
        sum_ = 0
        for n2 in hn2:
            sum_ = sum_ + torch.exp(torch.dist(n1, n2))
        cross_graph_msg = 0
        for n2 in hn2:
            cross_graph_msg += torch.exp(torch.dist(n1, n2))/sum_*n2
        result = torch.cat((result, n1 - cross_graph_msg.view(1, -1)),dim=0)
    for n1 in hn2:
        sum_ = 0
        for n2 in hn1:
            sum_ = sum_ + torch.exp(torch.dist(n1, n2))
        cross_graph_msg = 0
        for n2 in hn2:
            cross_graph_msg += torch.exp(torch.dist(n1, n2))/sum_*n2
        result = torch.cat((result, n1 - cross_graph_msg.view(1, -1)),dim=0)  
    return result

graph = torch.tensor(np.array(graph)).float()
edge = torch.tensor(np.array(edge)).float()
graph1 = torch.tensor(np.array(graph1)).float()
edge1 = torch.tensor(np.array(edge1)).float()
cross_graph_matching(graph,graph1)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
       

In [29]:
"""
The model 
"""


# Define a message layer, which is a MLP with one hidden layer, for each row of input it takes in a pair of node
# representations h1, h2 and an edge representation e12 between them and return a message from h2 to h1 via e12.
# The input is the concatenation of h1, h2, e12, over all such pairs of nodes. dim: d -> 2d -> 2d
class MessageLayer(nn.Module):
    def __init__(self, in_feats, edge_feats, hidden_feats, out_feats):
        super(MessageLayer, self).__init__()
        self.hidden = nn.Linear(in_feats + in_feats + edge_feats, hidden_feats)
        self.linear = nn.Linear(hidden_feats, out_feats)

    def forward(self, h):
        hi = F.relu(self.hidden(h))
        out = self.linear(hi)
        return out


# Define a node message receive layer, which has one GRU layer, the input is the concatenation of node representation h
# and the sum of all messages received by the node. dim: Each message is 2 * d, hence, input is 3 * d, output is d.
class NodeLayer(nn.Module):
    def __init__(self, in_feats):
        super(NodeLayer, self).__init__()
        self.hidden = nn.GRU(input_size=in_feats, hidden_size=in_feats // 3)

    def forward(self, h, msg):
        h_in = torch.cat((h, msg), dim=1)
        out = self.hidden(h_in.view((1,len(h_in),-1)))
        return out[-1][-1]


# Define a propagation layer, which has one MessageLayer on all pairs of nodes and one NodeLayer on all nodes.
# the input is h, a matrix containing all the node representations, each row is for one node, the edge feature matrix e.
class PropagationLayer(nn.Module):
    def __init__(self, in_feats, edge_feats, hidden_feats, msg_out_feats):
        super(PropagationLayer, self).__init__()
        self.message_layers = MessageLayer(in_feats, edge_feats, hidden_feats, msg_out_feats)
        self.node_layers = NodeLayer(in_feats + msg_out_feats)

    def forward(self, h, e):
        message_layer_input = match(h, e)
#         print(">>>>>>>>> msg layer in: ", message_layer_input.size())
        message_layer_output = self.message_layers(message_layer_input)
#         print(">>>>>>>>> msg layer out: ", message_layer_output)
#         print(">>>>>>>>> h: ", h)
        out = self.node_layers(h, compress(message_layer_output, compression_list))
#         print(">>>>>>>>> propagation result: ", out)
        return out


# Define the aggregator layer, which computes a graph level representation from the node representations after several
# propagation layers.
class AggregatorLayer(nn.Module):
    def __init__(self, in_feats, hidden_feats, graph_vector_feats):
        super(AggregatorLayer, self).__init__()
        self.linear1 = nn.Linear(in_feats, graph_vector_feats)    # this is the MLP
        self.linear2 = nn.Linear(in_feats, graph_vector_feats)    # this is MLP gate
        self.hidden3 = nn.Linear(graph_vector_feats, hidden_feats) # this is the hidden layer of MLP G
        self.linear3 = nn.Linear(hidden_feats, graph_vector_feats)  # this is the last layer of MLP G

    def forward(self, h):
        result_mlp = self.linear1(h)
        result_mlp_gate = torch.sigmoid(self.linear2(h))
        hidden = self.hidden3(result_mlp_gate*result_mlp)
        hidden = F.relu(torch.sum(hidden, dim=0))
        out = self.linear3(hidden)
#         print(">>>>>>>>>>>>>> out: ", out)
        return out


# Define the network model GNN, assume there are 3 prop layers
class GNN(nn.Module):
    def __init__(self, in_feats, edge_feats, msg_hidden_feats, msg_out_feats, aggregator_hidden_feats, graph_vector_feats):
        super(GNN, self).__init__()
        self.linear1 = nn.Linear(in_feats, in_feats)          # encoding of node features
        self.linear2 = nn.Linear(edge_feats, edge_feats)      # encoding of edge features
        self.prop1 = PropagationLayer(in_feats, edge_feats, msg_hidden_feats, msg_out_feats)
        self.prop2 = PropagationLayer(in_feats, edge_feats, msg_hidden_feats, msg_out_feats)
        self.prop3 = PropagationLayer(in_feats, edge_feats, msg_hidden_feats, msg_out_feats)
        self.aggregator = AggregatorLayer(in_feats, aggregator_hidden_feats, graph_vector_feats)

    # x is the raw node features, e is the raw
    def forward(self, x, e):
        h0 = self.linear1(x)
        e = self.linear2(e)
        h1 = self.prop1(h0, e)
        h2 = self.prop2(h1, e)
        h3 = self.prop3(h2, e)
#         print(">>>>>>>>>>>> output after last prop: ", h3)
        out = self.aggregator(h3)
        return out

In [30]:
"""
Loss function
"""
# the loss function of pair of graphs is max(0, gamma - t(1-d(G1,G2))), where gamma > 0 is a marginal parameter,
# d(G1,G2) = ||hG1-hG2||^2
gamma = 0.1                 # marginal parameter


def loss_function_pair(out1, out2, target):
    print(">>>>>>>>>>>>> out1:", out1)
    print(">>>>>>>>>>>>> out2:", out2)
    return torch.max(torch.zeros(1),0.1 - target * torch.dist(out1, out2))


# the loss function of triplet of graphs is max(0, (d(G1,G2) - d(G1,G3)+gamma)), where gamma > 0 is a marginal parameter,
# d(G1,G2) = ||hG1-hG2||^2, if G1 and G2 are closer than G1 and G3.
def loss_function_triplet(out1, out2, out3):
    return torch.max(torch.zeros(0),torch.dist(out1, out2) - torch.dist(out1, out3) + 0.1)

In [31]:
"""
Parameters
"""
d = 4
e = 4
in_feature = d
edge_feature = e
hidden_size = 2 * d
graph_vector_features = d // 2
# num_propagation = n

In [32]:
"""
Train the model
"""
num_epochs = 10
learning_rate = 0.01
target = 1
net1 = GNN(in_feature, edge_feature, 2*in_feature, 2*in_feature, graph_vector_features, graph_vector_features)
optimiser = torch.optim.SGD(net.parameters(), lr=learning_rate)
all_loss = []


def train(x1, e1, x2, e2, target):
    for epoch in range(num_epochs):
        G1_pred = net(x1, e1)
        G2_pred = net(x2, e2)
        loss = loss_function_pair(G1_pred, G2_pred, target)
        print(">>>>>>>>>>>>>> loss: ", loss)
        all_loss.append(loss)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))


graph1 = torch.tensor(np.array(graph)).float()
edge1 = torch.tensor(np.array(edge)).float()
print(">>>>>>>>> graph: ", graph1)
print(">>>>>>>>> edge: ", edge1)
        
train(graph1, edge1, graph1, edge1, target)

>>>>>>>>> graph:  tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
>>>>>>>>> edge:  tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])


TypeError: match() missing 2 required positional arguments: 'hn1' and 'edge1'